In [ ]:
import os
import arcpy
import pandas as pd
import numpy as np
import csv
from arcgis.features import GeoAccessor, GeoSeriesAccessor

arcpy.env.overwriteOutput = True

<h4>Modificación de la capa -Jurisdicción- a Municipio</h4>

In [ ]:
ruta_capa_jurisdiccion = r"D:\PUBLIC\PGAR\Resultados\4.PRO\PGAR\Local_BD_PGAR.gdb\_6_1_Jurisdiccion_CAR"
ruta_capa_geocodificacion = r"D:\PUBLIC\PGAR\Resultados\4.PRO\PGAR\Local_BD_PGAR.gdb\_6_2_Geocodificacion_Regionales"

# ? --- PREPARACIÓN DIRECCIÓN ---
df_direccion = pd.DataFrame.spatial.from_featureclass(ruta_capa_jurisdiccion)
# Se implementa .loc para evitar alertas en la modificación del dataframe
df_direccion_columnas_estandarizadas = df_direccion.loc[:,['Direccion']]
df_direccion_columnas_estandarizadas.rename({'Direccion':'nombre_direccion'}, axis = 1, inplace = True)
# Transformación de registros de mayúsculas a letra capital, esto por medio de la función capitalize del método str.
df_direccion_columnas_estandarizadas['nombre_direccion'] = df_direccion_columnas_estandarizadas['nombre_direccion'].str.capitalize()

# ** Generacion de DF, geocodificacion de regionales
df_geocodificacion = pd.DataFrame.spatial.from_featureclass(ruta_capa_geocodificacion)

# ? --- PREPARACIÓN MUNICIPIO ---
df_jurisdiccion = pd.DataFrame.spatial.from_featureclass(ruta_capa_jurisdiccion)
# Se implementa .loc para evitar alertas en la modificación del dataframe
df_jurisdiccion_columnas_estandarizadas = df_jurisdiccion.loc[:,['OBJECTID','Municipio','CODDANE','Departamen','Direccion','SHAPE']]
df_jurisdiccion_columnas_estandarizadas.rename({'OBJECTID':'codigo_municipio', 'Municipio':'nombre_municipio','CODDANE':'codigo_municipio_dane','Departamen':'nombre_departamento'}, axis = 1, inplace = True)
# Transformación de registros de mayúsculas a letra capital, esto por medio de la función capitalize del método str.
df_jurisdiccion_columnas_estandarizadas['nombre_municipio'] = df_jurisdiccion_columnas_estandarizadas['nombre_municipio'].str.capitalize()
df_jurisdiccion_columnas_estandarizadas['nombre_departamento'] = df_jurisdiccion_columnas_estandarizadas['nombre_departamento'].str.capitalize()
df_jurisdiccion_columnas_estandarizadas['Direccion'] = df_jurisdiccion_columnas_estandarizadas['Direccion'].str.capitalize()

<h4>Preparación tabla -Direccion-</h4>

In [ ]:
df_direccion_columnas_agrupadas = df_direccion_columnas_estandarizadas.groupby(['nombre_direccion']).agg({'nombre_direccion':'count'})

# Generación del campo incremental de acuerdo al número de capas
codigo_direccion = []

# Se suma uno por el header dentro del data frame
for numero_direcciones in range(1, df_direccion_columnas_agrupadas.shape[0]+1):
    codigo_direccion.append(numero_direcciones)
 
# Se utiliza el método .loc para incorporar la columna, que en este caso fue la lista que se creo previamente   
df_direccion_columnas_agrupadas.loc[:,'codigo_direccion'] = codigo_direccion

# Se cambia el nombre de las columnas asociadas a los conteos (funcion -rename-)
df_direccion_columnas_agrupadas.rename({'nombre_direccion':'conteo_nombres_direcciones'}, axis = 1, inplace = True)

# Se resetan los índices
df_tabla_direccion_completa = df_direccion_columnas_agrupadas.reset_index()

# Se estandariza la tabla final
df_direccion = df_tabla_direccion_completa[['codigo_direccion','nombre_direccion']]

# Merge de tabla dirección alfanumérica con su respectiva coordenada
df_unificado_geo = pd.merge(df_geocodificacion, df_direccion, how='inner',left_on='nombre_direccion', right_on='nombre_direccion')

# ** Preparación para salida final
df_direccion_final = df_unificado_geo.loc[:,['codigo_direccion','nombre_direccion','direccion','latlong','SHAPE']]

<h4>Creación y registro del campo -codigo_departamento-</h4>

In [ ]:
# Se utiliza el método .loc para incorporar la columna, que en este caso fue la lista que se creo previamente
# La columna creada se le registran los datos haciendo un substring a la columna -codigo_municipio-  
df_jurisdiccion_columnas_estandarizadas.loc[:,'codigo_departamento'] = df_jurisdiccion_columnas_estandarizadas['codigo_municipio_dane'].str.slice(0, 2)

<h4>Incorporación del campo -codigo_direccion- en la capa -Municipio-</h4>

In [ ]:
# Ejecución de un inner join sin where en ningún campo
df_tabla_municipio = pd.merge(df_jurisdiccion_columnas_estandarizadas, df_direccion, how='inner',left_on='Direccion', right_on='nombre_direccion')
df_tabla_municipio_estandarizado = df_tabla_municipio.loc[:,['codigo_municipio','codigo_municipio_dane','nombre_municipio','codigo_departamento', 'codigo_direccion','SHAPE']]

<h4>Exportar DF a Feature Class</h4>

In [ ]:
ruta_bd = r"D:\PUBLIC\PGAR\Resultados\4.PRO\PGAR\Local_BD_PGAR.gdb"

# --- PREPARACIÓN DIRECCION ---
nombre_tabla_direccion = 'direccion'
ruta_capa_direccion = os.path.join(ruta_bd, nombre_tabla_direccion)
df_direccion_final.spatial.to_featureclass(location=ruta_capa_direccion)
print("------------------------------ Se crea la capa -direccion- ------------------------------")

# --- PREPARACIÓN MUNICIPIO ---
nombre_capa_municipio = 'municipio_dissolve'
ruta_capa_municipio = os.path.join(ruta_bd, nombre_capa_municipio)
df_tabla_municipio_estandarizado.spatial.to_featureclass(location=ruta_capa_municipio)
print("------------------------------ Se crea la capa -municipio dissolve- ------------------------------")

<h4>Dissolve Municipio</h4>

In [ ]:
capa_municipio_dissolve = r"D:\PUBLIC\PGAR\Resultados\4.PRO\PGAR\Local_BD_PGAR.gdb\municipio_dissolve"
nombre_capa_salida_municipio = 'municipio'
ruta_capa_salida = os.path.join(ruta_bd,nombre_capa_salida_municipio)

arcpy.management.Dissolve(capa_municipio_dissolve, ruta_capa_salida, ['codigo_municipio', 'codigo_municipio_dane', 'nombre_municipio', 'codigo_departamento','codigo_direccion'], multi_part = 'MULTI_PART')
print("--------------- Capa Municipio Disuelta ----------------")